In [1]:
! pip install ortools

     ---------------------------------------- 40.5/40.5 MB 2.0 MB/s eta 0:00:00
     -------------------------------------- 903.8/903.8 KB 3.8 MB/s eta 0:00:00
     -------------------------------------- 126.7/126.7 KB 2.5 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


# Generator






In [3]:
#Generate sample test 
import random as rd 
def Generator(N):
  M = rd.randrange(int(0.4 * N), int(0.6 * N), 1)
  K = rd.randrange(int(0.2 * N), int(0.35 * N), 1)
  r1 = int(N/K)
  r2 = int(M/K)
  a = rd.randrange(int(0.8 * r1), r1, 1)
  b = rd.randrange(r1, int(r1 * 1.2), 1) 
  c = rd.randrange(int(0.8 * r2), r2, 1)
  d = rd.randrange(r2, int(r2 * 1.2), 1)
  #t = [0 for x in range(N + 1)]
  t = [x for x in range(1, N + 1)]
  rd.shuffle(t)
  for _ in range(N):
    t[_] %= M
    t[_] += 1
  s = [[0 for j in range(N)] for i in range(N)]
  for i in range(N):
    for j in range(i + 1, N):
      s[i][j] = rd.randrange(1, 15, 1)
      s[j][i] = s[i][j]
  g = [[0 for i in range(N)] for j in range(M)]
  for j in range(M):
    for i in range(N):
      g[j][i] = rd.randrange(1, 15, 1)
  e = rd.randrange(1, 15, 1)
  f = rd.randrange(1, 15, 1)
  with open('data size ' + str(N), 'w') as text:
    text.write(str(N) + ' ' + str(M) + ' ' + str(K) + '\n')
    text.write(str(a) + ' ' + str(b) + ' ' + str(c) + ' ' + str(d) + ' ' + str(e) + ' ' + str(f) + '\n')
    for i in range(N):
      for x in s[i]:
        text.write(str(x) + ' ')
      text.write('\n') 
    for j in range(M):
      for x in g[j]:
        text.write(str(x) + ' ')
      text.write('\n')
    for i in range(N):
      text.write(str(t[i] + ' ')) 



# Integer Linear Programming

In [ ]:
# LINEAR PROGRAM 
from ortools.linear_solver import pywraplp 
from time import perf_counter

# input data 
def input_data(filename):
  with open(filename) as text:
    [N, M, K] = [int(x) for x in text.readline().split()]
    [a, b, c, d, e, f] = [int(x) for x in text.readline().split()]
    s = [[0 for x in range(N + 1)]]
    for i in range(N):
      r = [int(x) for x in text.readline().split()]
      s.append([0] + r)
    g = [[0 for x in range(N + 1)]]
    for j in range(1, M + 1):
      r = [int(x) for x in text.readline().split()]
      g.append([0] + r) 
    t = [0] + [int(x) for x in text.readline().split()]
  return N, M, K, a, b, c, d, e, f, s, g, t 
filename = '/content/data_size_14.txt'
N, M, K, a, b, c, d, e, f, s, g, t = input_data(filename)

start = perf_counter()
# SET VARIABLES
solver = pywraplp.Solver.CreateSolver('CBC')
INF = solver.infinity()
z = [[0 for i in range(K + 1)]]

for i in range(1, N + 1):
  z.append([0] + [solver.IntVar(0, 1, 'z(' + str(i) + ',' + str(m) + ')') for m in range(1, K + 1)])


y = [[0 for i in range(K + 1)]]

for j in range(1, M + 1):
  y.append([0] + [solver.IntVar(0, 1, 'y(' + str(j) + ',' + str(m) + ')') for m in range(1, K + 1)])

# x(i1,i2,m)
x = [[[0 for m in range(K + 1)] for i2 in range(N + 1)] for i1 in range(N + 1)]
for i1 in range(1, N + 1):
  for i2 in range(1, N + 1):
    for m in range(1, K + 1):
      x[i1][i2][m] = solver.IntVar(0, 1, 'x(' + str(i1) + ',' + str(i2) + ',' + str(m) + ')')

# p(i,j,m)

p = [[[0 for m in range(K + 1)] for j in range(M + 1)] for i in range(N + 1)]
for i in range(1, N + 1):
  for j in range(1, M + 1):
    for m in range(1, K + 1):
      p[i][j][m] = solver.IntVar(0, 1, 'p(' + str(i) + ',' + str(j) + ',' + str(m) + ')')
# total 
total = solver.IntVar(0, INF, 'total')
# E, F 
E = solver.IntVar(e, INF, 'E')
F = solver.IntVar(f, INF, 'F')

# SET CONSTRAINTS

# constraint 1 
for m in range(1, K + 1):
  cstr = solver.Constraint(a, b)
  for i in range(1, N + 1):
    cstr.SetCoefficient(z[i][m], 1)

# constraint 2
for m in range(1, K + 1):
  cstr = solver.Constraint(c, d)
  for j in range(1, M + 1):
    cstr.SetCoefficient(y[j][m], 1)

# constraint 3 

for m in range(1, K + 1):
  for i in range(1, N + 1):
    cstr = solver.Constraint(0, 1)
    cstr.SetCoefficient(z[i][m], 1)
    cstr.SetCoefficient(y[t[i]][m], 1)

# constraint 4 
for m in range(1, K + 1):
  for i1 in range(1, N + 1):
    for i2 in range(i1+1, N + 1):
        cstr = solver.Constraint(0, s[i1][i2]/e + 1)
        cstr.SetCoefficient(z[i1][m], 1)
        cstr.SetCoefficient(z[i2][m], 1)

# constraint 5 
for m in range(1, K + 1):
  for i in range(1, N + 1):
    for j in range(t[i], M + 1):
        cstr = solver.Constraint(0, g[j][i]/f + 1)
        cstr.SetCoefficient(z[i][m], 1)
        cstr.SetCoefficient(y[j][m], 1)
        
# constraint 6
for i in range(1, N + 1):
  cstr = solver.Constraint(1, 1)
  for m in range(1, K + 1):
    cstr.SetCoefficient(z[i][m], 1)

# constraint 7 
for j in range(1, M + 1):
  cstr = solver.Constraint(1, 1)
  for m in range(1, K + 1):
    cstr.SetCoefficient(y[j][m], 1)

# constraint 8 
for m in range(1, K + 1):
  for i1 in range(1, N):
    for i2 in range(i1 + 1, N + 1): 
      cstr1 = solver.Constraint(0, INF)
      cstr1.SetCoefficient(z[i1][m], 1)
      cstr1.SetCoefficient(z[i2][m], 1)
      cstr1.SetCoefficient(x[i1][i2][m], -2) 

      cstr2 = solver.Constraint(-1, INF) 
      cstr2.SetCoefficient(x[i1][i2][m], 1)
      cstr2.SetCoefficient(z[i1][m], -1)
      cstr2.SetCoefficient(z[i2][m], -1)

      cstr3 = solver.Constraint(0, 0)
      cstr3.SetCoefficient(x[i1][i2][m], 1)
      cstr3.SetCoefficient(x[i2][i1][m], -1)

# constraint 9 
for m in range(1, K + 1):
  for i in range(1, N + 1):
    for j in range(1, M + 1):
      cstr1 = solver.Constraint(0, INF)
      cstr1.SetCoefficient(y[j][m], 1)
      cstr1.SetCoefficient(z[i][m], 1)
      cstr1.SetCoefficient(p[i][j][m], -2)

      cstr2 = solver.Constraint(-1, INF)
      cstr2.SetCoefficient(p[i][j][m], 1)
      cstr2.SetCoefficient(y[j][m], -1)
      cstr2.SetCoefficient(z[i][m], -1)
for i in range(1, N + 1):
  for m in range(1, K + 1):
    cstr = solver.Constraint(0, 0)
    cstr.SetCoefficient(p[i][t[i]][m], 1)
# constraint 10 
cstr10 = solver.Constraint(0, 0) 
cstr10.SetCoefficient(total, -1)
for m in range(1, K + 1):
  for i1 in range(1, N):
    for i2 in range(i1 + 1, N + 1): 
      cstr10.SetCoefficient(x[i1][i2][m], s[i1][i2])

for m in range(1, K + 1):
  for i in range(1, N + 1):
    for j in range(1, M + 1):
      cstr10.SetCoefficient(p[i][j][m], g[j][i])
# constraint 11: s(i1,i2)(N - (N - 1)*x(i1,i2,m)) >= E with i1 != i2
for i1 in range(1, N + 1):
  for i2 in range(i1+1, N + 1):
    for m in range(1, K + 1):
        cstr11 = solver.Constraint(e, s[i1][i2] * N)
        cstr11.SetCoefficient(x[i1][i2][m], s[i1][i2] * (N - 1))
        cstr11.SetCoefficient(E, 1)

# constraint 12: g(j, i)(N - (N - 1)p(i, j, m)) >= F
for i in range(1, N + 1):
  for j in range(1, M + 1):
    for m in range(1, K + 1):
      cstr12 = solver.Constraint(f, N * g[j][i])
      cstr12.SetCoefficient(p[i][j][m], g[j][i] * (N - 1))
      cstr12.SetCoefficient(F, 1)

#Objective 
alpha = 1 
beta = 1
gamma = 1
obj = solver.Objective()
obj.SetCoefficient(total, alpha)
obj.SetCoefficient(E, beta)
obj.SetCoefficient(F, gamma)
obj.SetMaximization()
rs = solver.Solve()
end = perf_counter()

# out put data

def print_solution():
  for m in range(1, K + 1):
      print('_________________')
      print('|COUNCIL: ' + '\t' + str(m) + '\t' + '|')
      print('_________________')
      for i in range(1, N + 1):
        if z[i][m].solution_value() == 1:
           print('|Student ' + '\t' + str(i) + '\t' + '|')
      print('_________________')
      for j in range(1, M + 1):
        if y[j][m].solution_value() == 1:
          print('|Teacher ' + '\t' + str(j) + '\t' + '|')
      print('_________________')
      print()
      print()
  print()
  print('____________________________________')
  print('Total similarity value: ' + '\t' + str(total.solution_value()))
  print('Value of e: ' + '\t' + str(E.solution_value()))
  print('Value of f: ' + '\t' + str(F.solution_value()))
  print('____________________________________')
  print()
print_solution()
print('Time :'+str(end-start),'s' +'\n' +'____________________________________')

# Constraint programming model

In [ ]:
#non-matrix model
from ortools.sat.python import cp_model

with open('/content/data_size_20.txt','r') as f :
  m = []
  for i in f:
    m.append(list (map(int, i.split())))
  [N,M,K] = m[0]
  [a, b, c, d, e, f] = m[1]
  t = m[-1]
  s = (m[2:2+N])
  g = (m[N+2:N+M+2])

model = cp_model.CpModel()

#variables
x_bin = [[model.NewIntVar(0,1, 'x_bin['+str(i)+']['+str(j)+']') for j in range(K)] for i in range(N)]#student binary
x_int = [model.NewIntVar(0,K-1, 'x_int['+str(u)+']') for u in range(N)]
y_bin = [[model.NewIntVar(0,1, 'y_bin['+str(m)+']['+str(n)+']') for n in range(K)] for m in range(M)]
y_int = [model.NewIntVar(0,K-1, 'y_int['+str(v)+']') for v in range(M)]


#constraints
#1
for j in range(K):
  model.Add(a <= sum([x_bin[i][j] for i in range(N)]))
  model.Add(b >= sum([x_bin[i][j] for i in range(N)]))

for n in range(K):
  model.Add(c <= sum([y_bin[i][j] for i in range(M)]))
  model.Add(d >= sum([y_bin[i][j] for i in range(M)]))

for i in range(N):
    for j in range(K):
        b = model.NewBoolVar('b')
        model.Add(x_bin[i][j] == 1).OnlyEnforceIf(b)
        model.Add(x_bin[i][j] == 0).OnlyEnforceIf(b.Not())
        model.Add(x_int[i] == j).OnlyEnforceIf(b)
        model.Add(x_int[i] != j).OnlyEnforceIf(b.Not())

for m in range(M):
    for n in range(K):
        c = model.NewBoolVar('c')
        model.Add(y_bin[m][n] == 1).OnlyEnforceIf(c)
        model.Add(y_bin[m][n] == 0).OnlyEnforceIf(c.Not())
        model.Add(y_int[m] == n).OnlyEnforceIf(c)
        model.Add(y_int[m] != n).OnlyEnforceIf(c.Not())

#2: teacher != student
for i in range (N):
  model.Add(x_int[i] != y_int[t[i]-1])

#3: similarity

for room in range(K):
  for i in range(N-1):
    for j in range(i+1, N):
      bo = model.NewBoolVar('bo')
      model.Add(x_int[i] == x_int[j]).OnlyEnforceIf(bo)
      model.Add(x_int[i] != x_int[j]).OnlyEnforceIf(bo.Not())
      model.Add(e <= s[i][j]).OnlyEnforceIf(bo)

for room in range(K):
  for m in range(M):
    for i in range(N):
      boo = model.NewBoolVar('boo')
      model.Add(y_int[m] == x_int[i]).OnlyEnforceIf(boo)
      model.Add(y_int[m] != x_int[i]).OnlyEnforceIf(boo.Not())
      model.Add(f <= g[m][i]).OnlyEnforceIf(boo)

solver = cp_model.CpSolver()
status = solver.Solve(model)

class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    #print intermediate solution
    def __init__(self, var1, var2):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__var1 = var1
        self.__var2 = var2
        self.__solution_count = 0
    def on_solution_callback(self):
        self.__solution_count += 1
        print ('solution #', self.__solution_count)
        print ('   student assign:')
        for i in range(len(self.__var1)):        
          print('      student',i+1, 'in room', self.Value(self.__var1[i])+1)
        print ('   professor assign:')
        for i in range(len(self.__var2)):        
          print('      professor',i+1, 'in room', self.Value(self.__var2[i])+1)
    def solution_count(self):
        return self.__solution_count

solution_printer = VarArraySolutionPrinter(x_int, y_int)
solver.SearchForAllSolutions(model, solution_printer)
print(f'Number of solution:', solution_printer.solution_count())



# Brutce Force

In [ ]:
from itertools import combinations
from time import perf_counter
def input_data(filename):
    with open(filename) as text:
        [N, M, K] = [int(x) for x in text.readline().split()]
        [a, b, c, d, e, f] = [int(x) for x in text.readline().split()]
        s = [[0 for x in range(N + 1)]]
        for i in range(N):
            r = [int(x) for x in text.readline().split()]
            s.append([0] + r)
        g = [[0 for x in range(N + 1)]]
        for j in range(1, M + 1):
            r = [int(x) for x in text.readline().split()]
            g.append([0] + r) 
        t = [0] + [int(x) for x in text.readline().split()]
    return N, M, K, a, b, c, d, e, f, s, g, t 

N, M, K, a, b, c, d, e, f, s, g, t = input_data('/content/data_size_20.txt')
start = perf_counter()
class Council(object):
    global s, g 
    def __init__(self, m, teacher = [], student = []):
        self.Teacher = teacher # contains index of teacher in the council
        self.Student = student # contains index of teacher in the council
        self.Order = m          # the order of the council
        self.Pre_Council = None 
    def similarity_value_ss(self): # the total similarity value between each pair of projects 
        total_ss = 0
        for i in range(len(self.Student) - 1):
            for j in range(i + 1, len(self.Student)):
                total_ss += s[self.Student[i]][self.Student[j]] 
        return total_ss
    def similarity_value_ts(self): # the total similarity value between each pair of teachers and students
        total_ts = 0
        for j in self.Teacher:
            for i in self.Student:
                total_ts += g[j][i]
        return total_ts
    def total_similarity(self):
        return self.similarity_value_ts() + self.similarity_value_ss()
    def e_value(self): # return the minimum similarity value of each pair of students
        e = float('inf') 
        for i in range(len(self.Student) - 1):
            for j in range(i + 1, len(self.Student)):
                e = min(e, s[self.Student[i]][self.Student[j]])
        return e 
    def f_value(self): # return the minimum similarity value between teacher and student in the council
        f = float('inf') 
        for j in self.Teacher:
            for i in self.Student:
                f = min(f, g[j][i])
        return f 
    def __str__(self): # return the name of the council and its teachers and students 
        string = 'Council: ' + '\t' + str(self.Order) + '\n' + ' ' + '\n'
        for x in self.Student:
            string += 'Student ' + '\t' + str(x) + '\n'
        string += '____________________' + '\n'
        for y in self.Teacher:
            string += 'Teacher ' + '\t' + str(y) + '\n'
        string += '   ' + '\n'
        return string 

#set variables 
current_solution = [0 for x in range(K + 1)] # list containt councils 
optimal_solution = [0 for x in range(K + 1)] 
opt_value = -float('inf') 

def Subsets_of_set(lst: list, num): # return subset with length 'num' of the given list
    set_of_ids = combinations(range(len(lst)), num)
    list_of_subsets = []
    for id in set_of_ids:
        subset = []
        for i in id:
            subset.append(lst[i])
        list_of_subsets.append(sorted(subset))
    return list_of_subsets
def check_valid_council(C: Council):
    for j in C.Teacher:
        for i in C.Student:
            if j == t[i]:
                return False
    if a <= len(C.Student) <= b and c <= len(C.Teacher) <= d and C.e_value() >= e and C.f_value() >= f:
        return True 
    else:
        return False
        
def Next_Councils(m, current_solution):
  if m == K:
    return []
  list_student = [x for x in range(1, N + 1)]
  list_teacher = [x for x in range(1, M + 1)]
  for i in range(1, m + 1):
    for student in current_solution[i].Student:
      list_student.remove(student)
    for teacher in current_solution[i].Teacher:
      list_teacher.remove(teacher)        
  if m == K - 1:
    C = Council(m + 1)
    C.Teacher = list_teacher[:]
    C.Student = list_student[:]
    if check_valid_council(C):
      return [C]
    else:
      return []
  if len(list_student) >= a * (K - m) and len(list_teacher) >= c * (K - m):
    rs = []
    max_student = len(list_student) - a * (K - m - 1)
    max_student = min(max_student, b)
    max_teacher = len(list_teacher) - c * (K - m - 1)
    max_teacher = min(max_teacher, d)
    for num_student in range(a, max_student + 1):
      for group_of_students in Subsets_of_set(list_student, num_student):
        C1 = Council(0, [], group_of_students)
        if C1.e_value() < e: 
          continue 
        for num_teacher in range(c, max_teacher + 1):
          for group_of_teachers in Subsets_of_set(list_teacher, num_teacher):
            C2 = Council(0, group_of_teachers, [])
            if C2.f_value() < f:
              continue 
            rs.append(Council(m + 1, group_of_teachers, group_of_students))
    return [x for x in rs if check_valid_council(x)] 
  else:
    return []
def total_similarity_of_solution(solution: list):
    total = 0
    for x in solution:
        if x != 0:
            total += x.total_similarity()
    return total 
def printSolution():
    for m in range(1, K + 1):
        print(optimal_solution[m])
    print('Optimal similarity value: ', opt_value)
def Try(m):
    global current_solution, opt_value, optimal_solution 
    Candidate = Next_Councils(m - 1, current_solution)[:]
    for y in Candidate:
        current_solution[m] = y 
      
        if m == K:
            if total_similarity_of_solution(current_solution) > opt_value:
                opt_value = total_similarity_of_solution(current_solution)
                optimal_solution = current_solution[:]    
        else:
            Try(m + 1)
        current_solution[m] = 0
        
Try(1)
end= perf_counter()
printSolution()
print('time',end-start)

Council: 	1
 
Student 	1
Student 	2
Student 	4
Student 	11
____________________
Teacher 	5
Teacher 	7
Teacher 	8
   

Council: 	2
 
Student 	3
Student 	7
Student 	16
Student 	17
____________________
Teacher 	3
Teacher 	6
   

Council: 	3
 
Student 	5
Student 	8
Student 	12
Student 	15
____________________
Teacher 	1
Teacher 	4
Teacher 	12
   

Council: 	4
 
Student 	6
Student 	10
Student 	13
Student 	14
____________________
Teacher 	2
Teacher 	9
Teacher 	13
   

Council: 	5
 
Student 	9
Student 	18
Student 	19
Student 	20
____________________
Teacher 	10
Teacher 	11
   

Optimal similarity value:  642
time 2.421738873999857


# Brand and Bound

In [ ]:
from itertools import combinations
from time import perf_counter
from math import comb


def input_data(filename):
    with open(filename) as text:
        [N, M, K] = [int(x) for x in text.readline().split()]
        [a, b, c, d, e, f] = [int(x) for x in text.readline().split()]
        s = [[0 for x in range(N + 1)]]
        for i in range(N):
            r = [int(x) for x in text.readline().split()]
            s.append([0] + r)
        g = [[0 for x in range(N + 1)]]
        for j in range(1, M + 1):
            r = [int(x) for x in text.readline().split()]
            g.append([0] + r) 
        t = [0] + [int(x) for x in text.readline().split()]
    return N, M, K, a, b, c, d, e, f, s, g, t 

N, M, K, a, b, c, d, e, f, s, g, t = input_data('data_size_40.txt')
start = perf_counter()
class Council(object):
    global s, g 
    def __init__(self, m, teacher = [], student = []):
        self.Teacher = teacher # contains index of teacher in the council
        self.Student = student # contains index of teacher in the council
        self.Order = m          # the order of the council
        self.Pre_Council = None 
    def similarity_value_ss(self): # the total similarity value between each pair of projects 
        total_ss = 0
        for i in range(len(self.Student) - 1):
            for j in range(i + 1, len(self.Student)):
                total_ss += s[self.Student[i]][self.Student[j]] 
        return total_ss
    def similarity_value_ts(self): # the total similarity value between each pair of teachers and students
        total_ts = 0
        for j in self.Teacher:
            for i in self.Student:
                total_ts += g[j][i]
        return total_ts
    def total_similarity(self):
        return self.similarity_value_ts() + self.similarity_value_ss()
    def e_value(self): # return the minimum similarity value of each pair of students
        e = float('inf') 
        for i in range(len(self.Student) - 1):
            for j in range(i + 1, len(self.Student)):
                e = min(e, s[self.Student[i]][self.Student[j]])
        return e 
    def f_value(self): # return the minimum similarity value between teacher and student in the council
        f = float('inf') 
        for j in self.Teacher:
            for i in self.Student:
                f = min(f, g[j][i])
        return f 
    def __str__(self): # return the name of the council and its teachers and students 
        string = 'Council: ' + '\t' + str(self.Order) + '\n' + ' ' + '\n'
        for x in self.Student:
            string += 'Student ' + '\t' + str(x) + '\n'
        string += '____________________' + '\n'
        for y in self.Teacher:
            string += 'Teacher ' + '\t' + str(y) + '\n'
        string += '   ' + '\n'
        return string 

#set variables 


def Subsets_of_set(lst: list, num): # return subset with length 'num' of the given list
    set_of_ids = combinations(range(len(lst)), num)
    list_of_subsets = []
    for id in set_of_ids:
        subset = []
        for i in id:
            subset.append(lst[i])
        list_of_subsets.append(sorted(subset))
    return list_of_subsets
def check_valid_council(C: Council):
    for j in C.Teacher:
        for i in C.Student:
            if j == t[i]:
                return False
    if a <= len(C.Student) <= b and c <= len(C.Teacher) <= d and C.e_value() >= e and C.f_value() >= f:
        return True 
    else:
        return False
        
def Next_Councils(m, current_solution):
    if m == K:
        return []
    list_student = [x for x in range(1, N + 1)]
    list_teacher = [x for x in range(1, M + 1)]
    for i in range(1, m + 1):
        for student in current_solution[i].Student:
            list_student.remove(student)
        for teacher in current_solution[i].Teacher:
            list_teacher.remove(teacher)        
    if m == K - 1:
        C = Council(m + 1)
        C.Teacher = list_teacher[:]
        C.Student = list_student[:]
        if check_valid_council(C):
            return [C]
        else:
            return []
    if len(list_student) >= a * (K - m) and len(list_teacher) >= c * (K - m):
        rs = []
        max_student = len(list_student) - a * (K - m - 1)
        max_student = min(max_student, b)
        max_teacher = len(list_teacher) - c * (K - m - 1)
        max_teacher = min(max_teacher, d)
        for num_student in range(a, max_student + 1):
            for group_of_students in Subsets_of_set(list_student, num_student):
                C1 = Council(0, [], group_of_students)
                if C1.e_value() < e: 
                    continue 
                for num_teacher in range(c, max_teacher + 1):
                    for group_of_teachers in Subsets_of_set(list_teacher, num_teacher):
                        C2 = Council(0, group_of_teachers, [])
                        if C2.f_value() < f:
                            continue 
                        rs.append(Council(m + 1, group_of_teachers, group_of_students))
        tmp1 = [x for x in rs if check_valid_council(x)] 
        dict = {x.total_similarity(): x for x in tmp1}
        annswer = []
        for key in sorted(dict.keys()):
            annswer.append(dict[key])
        annswer.reverse()
        return annswer
    else:
        return []
def total_similarity_of_solution(solution: list):
    total = 0
    for x in solution:
        if x != 0:
            total += x.total_similarity()
    return total 


current_solution = [0 for x in range(K + 1)] # list containt councils 
optimal_solution = [0 for x in range(K + 1)] 
opt_value = -float('inf') 
upper_bound = -float('inf')

def expected(m, current_solution):
    total1 = total_similarity_of_solution(current_solution)
    list_of_student = [x for x in range(1, N + 1)]
    list_of_teacher = [x for x in range(1, M + 1)]
    for i in range(1, m + 1):
        for student in current_solution[i].Student:
            list_of_student.remove(student)
        for teacher in current_solution[i].Teacher:
            list_of_teacher.remove(teacher)
    #find maximum similarity value in each pair of among students and each pair of teachers and students
    max_ss = 0
    for i1 in list_of_student:
        for i2 in list_of_student:
            if i1 != i2:
                max_ss = max(max_ss, s[i1][i2])
    max_ts = 0
    for i in list_of_student:
        for j in list_of_teacher:
            max_ts = max(max_ts, g[j][i])
    number_of_teacher_in_each_council = int(len(list_of_teacher)/(K - m))
    number_of_student_in_each_council = int(len(list_of_student)/(K - m))
    total2 = (K - m) * comb(number_of_student_in_each_council, 2) * max_ss + (K - m) * number_of_student_in_each_council * number_of_teacher_in_each_council * max_ts
    return total1 + total2   
def min_e_value_of_solution(solution):
    e_ = float('inf')
    for i in range(1, K + 1):
        e_ = min(solution[i].e_value(), e_)
    return e_ 
def min_f_value_of_solution(solution):
    f_ = float('inf')
    for i in range(1, K + 1):
        f_ = min(f_, solution[i].f_value())
    return f_


def Try(m):
    global current_solution, opt_value, optimal_solution, upper_bound 
    Candidate = Next_Councils(m - 1, current_solution)[:]
    for y in Candidate:
        current_solution[m] = y 
        if m <= K - 2:
            if upper_bound > expected(m, current_solution):
                current_solution[m] = 0
                continue
        if m == K:
            if total_similarity_of_solution(current_solution) > opt_value:
                opt_value = total_similarity_of_solution(current_solution)
                optimal_solution = current_solution[:]  
                upper_bound = opt_value
            elif total_similarity_of_solution(current_solution) == opt_value:
                total1 = min_e_value_of_solution(optimal_solution) + min_f_value_of_solution(optimal_solution)
                total2 = min_f_value_of_solution(current_solution) + min_e_value_of_solution(current_solution)
                if total1 < total2:
                    optimal_solution = current_solution[:]  
        else:
            Try(m + 1)
        current_solution[m] = 0
def printSolution():
    print('The optimal total value of similarity: ', opt_value)
    print('The maximum e_value: ', min_e_value_of_solution(optimal_solution))
    print('The maximum f_value: ', min_f_value_of_solution(optimal_solution))
    for m in range(1, K + 1):
        print(optimal_solution[m])
    
def Run():
    Try(1)
    printSolution()
Run()



#Constraint Programming Ver2

In [ ]:
from ortools.sat.python import cp_model

with open('data.txt','r') as f :
  m = []
  for i in f:
    m.append(list (map(int, i.split())))
  [N,M,K] = m[0]
  [a, b, c, d, e, f] = m[1]
  t = m[-1]
  s = (m[2:2+N])
  g = (m[N+2:N+M+2])

model = cp_model.CpModel()

#variables
x_bin = [[model.NewIntVar(0,1, 'x_bin['+str(i)+']['+str(j)+']') for j in range(K)] for i in range(N)]#student binary
x_int = [model.NewIntVar(0,K-1, 'x_int['+str(u)+']') for u in range(N)]
y_bin = [[model.NewIntVar(0,1, 'y_bin['+str(m)+']['+str(n)+']') for n in range(K)] for m in range(M)]
y_int = [model.NewIntVar(0,K-1, 'y_int['+str(v)+']') for v in range(M)]

student = [[model.NewIntVar(0,1, 'student['+str(i)+']['+str(j)+']') for j in range(N)] for i in range(N)]
prof = [[model.NewIntVar(0,1, 'prof['+str(m)+']['+str(n)+']') for n in range(N)] for m in range(M)]

for j in range(N):
  for i in range(N):
      model.Add(student [i][j] == student[j][i] )
      if i==j:
          model.Add(student [i][j] == 0)

#constraints
#1
for j in range(K):
  model.Add(a <= sum([x_bin[i][j] for i in range(N)]))
  model.Add(b >= sum([x_bin[i][j] for i in range(N)]))

for n in range(K):
  model.Add(c <= sum([y_bin[i][j] for i in range(M)]))
  model.Add(d >= sum([y_bin[i][j] for i in range(M)]))

for i in range(N):
    for j in range(K):
        b = model.NewBoolVar('b')
        model.Add(x_bin[i][j] == 1).OnlyEnforceIf(b)
        model.Add(x_bin[i][j] == 0).OnlyEnforceIf(b.Not())
        model.Add(x_int[i] == j).OnlyEnforceIf(b)
        model.Add(x_int[i] != j).OnlyEnforceIf(b.Not())

for m in range(M):
    for n in range(K):
        c = model.NewBoolVar('c')
        model.Add(y_bin[m][n] == 1).OnlyEnforceIf(c)
        model.Add(y_bin[m][n] == 0).OnlyEnforceIf(c.Not())
        model.Add(y_int[m] == n).OnlyEnforceIf(c)
        model.Add(y_int[m] != n).OnlyEnforceIf(c.Not())

#2: teacher != student
for i in range (N):
  model.Add(x_int[i]!=y_int[t[i]-1])

#3: similarity

E = model.NewIntVar(0,20, 'E')
F = model.NewIntVar(0,20, 'F')

for i in range(N-1):
  for j in range(i+1, N):
    bo = model.NewBoolVar('bo')
    model.Add(x_int[i]==x_int[j]).OnlyEnforceIf(bo)
    model.Add(x_int[i]!=x_int[j]).OnlyEnforceIf(bo.Not())
    model.Add(e<=s[i][j]).OnlyEnforceIf(bo)
    model.Add(student[i][j] == 1).OnlyEnforceIf(bo)
    model.Add(student[i][j] == 0).OnlyEnforceIf(bo.Not())
    model.Add(E<=s[i][j]).OnlyEnforceIf(bo)

for m in range(M):
  for i in range(N):
    boo = model.NewBoolVar('boo')
    model.Add(y_int[m]==x_int[i]).OnlyEnforceIf(boo)
    model.Add(y_int[m]!=x_int[i]).OnlyEnforceIf(boo.Not())
    model.Add(f<=g[m][i]).OnlyEnforceIf(boo)
    model.Add(prof[m][i]==1).OnlyEnforceIf(boo)
    model.Add(prof[m][i]==0).OnlyEnforceIf(boo.Not())
    model.Add(F<=g[m][i]).OnlyEnforceIf(boo)

objective_terms = []
for i in range(N-1):
    for j in range(i+1,N):
        objective_terms.append(s[i][j] * student[i][j])

for i in range(M):
    for j in range(N):
        objective_terms.append(g[i][j] * prof[i][j])


#set parameter
alpha = 1
beta = 1
gamma = 1

model.Maximize(alpha*sum(objective_terms)   +   beta*E  +   gamma*F)

      
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Total similarity = {solver.ObjectiveValue()}\n')
    print('E :',solver.Value(E))
    print('F :',solver.Value(F))

#RANDOM GEN

In [ ]:
import math
import random as rd
N, M, K = map(int, input().split())
(a, b) = (math.floor(N/K), math.ceil(N/K))
(c, d) = (math.floor(M/K), math.ceil(M/K))
(e,f) = (1,1)
t = [x for x in range(1, N + 1)]
rd.shuffle(t)
  
for _ in range(N):
    t[_] %= M
    t[_] += 1 


s = [[0 for _ in range(N)]for _ in range(N)]
g = [[0 for _ in range(N)]for _ in range(M)]

for i in range(N-1):
    for j in range(i+1, N):
        s[i][j] = rd.choice([1,1,1,1,5,6,6,7,7,7,8,8,9])
        s[j][i] = s[i][j]

for i in range(M):
    for j in range (N):
        g[i][j] = rd.choice ([1,1,1,1,5,6,6,7,7,7,8,8,9])

with open('data_size_' + str(N) + '.txt', 'w') as file:
    file.write(str(N) + ' ' + str(M) + ' ' + str(K) + '\n')
    file.write(str(a) + ' ' + str(b) + ' ' + str(c) + ' ' + str(d) + ' ' + str(e) + ' ' + str(f) + '\n')
    for i in range(N):
      for x in s[i][0:]:
        file.write(str(x) + ' ')
      file.write('\n') 
    for j in range(M):
      for x in g[j][0:]:
        file.write(str(x) + ' ')
      file.write('\n')
    for i in range(N):
      file.write(str(t[i]) + ' ')

20 15 5 


# Greedy method


In [ ]:
with open('data_size_20.txt','r') as f :
  m = []
  for i in f:
    m.append(list (map(int, i.split())))
  [N,M,K] = m[0]
  [a, b, c, d, e, f] = m[1]
  t = m[-1]
  s = (m[2:2+N])
  g = (m[N+2:N+M+2])

#student divide
student_list = [x+1 for x in range(N)]
divide = [[[],[]] for _ in range(K)]
for room in range(K):
  x = student_list[0]-1
  divide[room][0].append(student_list.pop(0))
  queue = []
  for i in student_list:
    if int(s[x][i-1]) > 1:
      queue.append(i)
  queue.sort(reverse=True)
  for i in range(min(len(queue),a-1)):
    divide[room][0].append(queue[i])
    student_list.remove(queue[i])


for room in range(K):
  if len(divide[room][0])<b and len(student_list)!=0:
    if s[divide[room][0][0]-1][student_list[0]-1]>1:
      divide[room][0].append(student_list.pop(0))
for room in range(K):
  if len(divide[room][0])<b and len(student_list)!=0:
      divide[room][0].append(student_list.pop(0))


#prof divide:
prof_list = [x+1 for x in range(M)]
for room in range(K):
  cand = []
  for prof in prof_list:
    if g[prof-1][divide[room][0][0]-1]>1:
      cand.append(prof)
  cand.sort(reverse=True)
  for i in range(min(len(cand),c)):
    divide[room][1].append(cand[i])
    prof_list.remove(cand[i])

for room in range(K):
  if len(divide[room][1])<d and len(prof_list)!=0:
    if g[prof_list[0]-1][divide[room][0][0]-1]>1:
      divide[room][1].append(prof_list.pop(0))
for room in range(K):
  if len(divide[room][1])<d and len(prof_list)!=0:
      divide[room][1].append(prof_list.pop(0))

sum = 0
for room in range(K):
  for i in range(len(divide[room][0])-1):
    for j in range(i+1,len(divide[room][0])):
      sum+= s[divide[room][0][i]-1][divide[room][0][j]-1]
  for i in range(len(divide[room][0])):
    for j in range(len(divide[room][1])):
      sum+= g[divide[room][1][j]-1][divide[room][0][i]-1]
def printSolution():
  string = ''
  for i in range(K):
    string += 'Council' + '\t' + str(i + 1) + '\n' + '______________' + '\n'
    for student in sorted(divide[i][0]):
      string += 'Student' + '\t' + str(student) + '\n'
    string += '______________' + '\n'
    for teacher in sorted(divide[i][1]):
      string += 'Teacher' + '\t' + str(teacher) + '\n'
    string += '\n' + '\n'
  print(string) 
  print('Total value of similarity: ' + '\t' + str(sum))
  
  min_e = float('inf')
  min_f = float('inf')

  for m in range(K):
    for student1 in divide[m][0]:
      for student2 in divide[m][0]:
        if student1 != student2:
          min_e = min(min_e, s[student1 - 1][student2 - 1])
    for student in divide[m][0]:
      for teacher in divide[m][1]:
        min_f = min(min_f, g[teacher - 1][student - 1])
  print('The e_value: ', min_e)
  print('The f_value: ', min_f)

printSolution()
